# Shallow autoencoder

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Libraries import

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras.layers import Dense, Dropout
from keras.models import Model, Sequential
from keras.layers import Dense
import tensorflow as tf
import time

## Arrhythmia

**Dataset source**: http://odds.cs.stonybrook.edu/arrhythmia-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources**:

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

F. Keller, E. Muller, K. Bohm.“HiCS: High-contrast subspaces for density-based outlier ranking.” ICDE, 2012.

In [5]:
data = pd.read_csv('./arrhythmia.csv', sep = ',')

In [6]:
# dropping columns that consist only of 0's
data = data.drop(columns = ['Col15', 'Col63', 'Col65', 'Col79', 'Col127', 'Col128','Col135', 'Col137', 'Col139','Col141',
'Col147', 'Col152', 'Col153','Col160','Col200', 'Col260', 'Col270'])

In [7]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col265,Col266,Col267,Col268,Col269,Col271,Col272,Col273,Col274,y
0,75.0,0.0,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,-0.3,0.0,9.0,-0.9,0.0,0.9,2.9,23.3,49.4,1
1,56.0,1.0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,-0.5,0.0,8.5,0.0,0.0,0.2,2.1,20.4,38.8,0
2,54.0,0.0,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.9,0.0,9.5,-2.4,0.0,0.3,3.4,12.3,49.0,0
3,55.0,0.0,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.1,0.0,12.2,-2.2,0.0,0.4,2.6,34.6,61.6,0
4,75.0,0.0,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,-0.4,0.0,13.1,-3.6,0.0,-0.1,3.9,25.4,62.8,1


In [8]:
data.shape

(452, 258)

In [9]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,386
1,66


### Autoencoders

In [10]:
features = data.drop(columns = ['y'])
target = data['y']

In [11]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [12]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(128,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [13]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [14]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=64,
)

end = time.process_time()
arrhythmia_autoencoders_train_time = end - start
print(end - start)


start = time.process_time()
threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
arrhythmia_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
8/8 [==============================] - 1s 3ms/step - loss: 0.0821 - mse: 0.1402
Epoch 2/20
8/8 [==============================] - 0s 2ms/step - loss: 0.0497 - mse: 0.0816
Epoch 3/20
8/8 [==============================] - 0s 1ms/step - loss: 0.0211 - mse: 0.0350
Epoch 4/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0117 - mse: 0.0217
Epoch 5/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0102 - mse: 0.0197
Epoch 6/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0100 - mse: 0.0193
Epoch 7/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0096 - mse: 0.0187
Epoch 8/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0094 - mse: 0.0184
Epoch 9/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0092 - mse: 0.0180
Epoch 10/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0090 - mse: 0.0175
Epoch 11/20
8/8 [==============================] - 0s 3ms/step - loss: 0.0089 - mse: 0.01

In [15]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
arrhythmia_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.8027162819908934

In [16]:
arrhythmia_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91       386
           1       0.49      0.41      0.45        66

    accuracy                           0.85       452
   macro avg       0.70      0.67      0.68       452
weighted avg       0.84      0.85      0.85       452



In [17]:
print(arrhythmia_autoencoders_report['1']['precision'])
print(arrhythmia_autoencoders_report['1']['recall'])
print(arrhythmia_autoencoders_report['1']['f1-score'])

0.4909090909090909
0.4090909090909091
0.4462809917355372


In [18]:
precision, recall, thresholds = precision_recall_curve(target, scores)
arrhythmia_ae_auc_precision_recall = metrics.auc(recall, precision)
print(arrhythmia_ae_auc_precision_recall)

0.44617900901738616


## Cardiotocography

**Dataset source**: http://odds.cs.stonybrook.edu/cardiotocogrpahy-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [19]:
data = pd.read_csv('./Cardiotocography.csv')

In [20]:
data.shape

(1831, 22)

In [21]:
data['y'] = data['y'].astype(int)

In [22]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,1655
1,176


In [23]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21,y
0,0.004912,0.693191,-0.203640,0.595322,0.353190,-0.061401,-0.278295,-1.650444,0.759072,-0.420487,...,-0.798376,1.854728,0.622631,0.963083,0.301464,0.193113,0.231498,-0.289786,-0.493294,0
1,0.110729,-0.079903,-0.203640,1.268942,0.396246,-0.061401,-0.278295,-1.710270,0.759072,-0.420487,...,-0.798376,1.854728,0.278625,0.963083,0.301464,0.129265,0.093563,-0.256385,-0.493294,0
2,0.216546,-0.272445,-0.203640,1.050988,0.148753,-0.061401,-0.278295,-1.710270,1.106509,-0.420487,...,-1.332931,0.314688,2.342663,-0.488279,0.061002,0.065417,0.024596,-0.256385,1.140018,0
3,0.004912,0.727346,-0.203640,1.212171,-0.683598,-0.061401,-0.278295,-1.710270,1.106509,-0.420487,...,-1.332931,0.314688,1.654650,-0.488279,0.061002,0.193113,0.093563,-0.323186,1.140018,0
4,-0.100905,0.363595,1.321366,1.027120,0.141359,-0.061401,-0.278295,-0.992364,-0.051613,-0.420487,...,-0.085638,-0.565334,0.278625,-0.488279,-0.059229,0.065417,0.024596,-0.456787,1.140018,0


### Autoencoders

In [24]:
features = data.drop(columns = ['y'])
target = data['y']

In [25]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [26]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(10,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [27]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [28]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=64,
)

end = time.process_time()
cardio_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
cardio_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
29/29 [==============================] - 1s 2ms/step - loss: 0.0711 - mse: 0.1191
Epoch 2/20
29/29 [==============================] - 0s 3ms/step - loss: 0.0613 - mse: 0.1010
Epoch 3/20
29/29 [==============================] - 0s 2ms/step - loss: 0.0469 - mse: 0.0763
Epoch 4/20
29/29 [==============================] - 0s 2ms/step - loss: 0.0328 - mse: 0.0540
Epoch 5/20
29/29 [==============================] - 0s 4ms/step - loss: 0.0244 - mse: 0.0415
Epoch 6/20
29/29 [==============================] - 0s 3ms/step - loss: 0.0203 - mse: 0.0354
Epoch 7/20
29/29 [==============================] - 0s 2ms/step - loss: 0.0184 - mse: 0.0329
Epoch 8/20
29/29 [==============================] - 0s 2ms/step - loss: 0.0178 - mse: 0.0321
Epoch 9/20
29/29 [==============================] - 0s 2ms/step - loss: 0.0171 - mse: 0.0310
Epoch 10/20
29/29 [==============================] - 0s 2ms/step - loss: 0.0164 - mse: 0.0298
Epoch 11/20
29/29 [==============================] - 0s 3ms/step - lo

In [29]:
confusion_matrix(target, predictions)

array([[1518,  137],
       [  66,  110]], dtype=int64)

In [30]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
cardio_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.8985443559461687

In [31]:
cardio_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94      1655
           1       0.45      0.62      0.52       176

    accuracy                           0.89      1831
   macro avg       0.70      0.77      0.73      1831
weighted avg       0.91      0.89      0.90      1831



In [32]:
precision, recall, thresholds = precision_recall_curve(target, scores)
cardio_ae_auc_precision_recall = metrics.auc(recall, precision)
print(cardio_ae_auc_precision_recall)

0.5615925582712333


## ForestCover

**Dataset source**: http://odds.cs.stonybrook.edu/forestcovercovertype-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

Kai Ming Ting, Guang-Tong Zhou, Fei Tony Liu & Tan Swee Chuan. (2010). Mass Estimation and Its Applications. Proceedings of The 16th ACM SIGKDD Conference on Knowledge Discovery and Data Mining 2010. pp. 989-998.

Swee Chuan Tan, Kai Ming Ting & Fei Tony Liu. (2011). Fast Anomaly Detection for Streaming Data. Proceedings of the International Joint Conference on Artificial Intelligence 2011. pp.1151-1156.

In [33]:
data = pd.read_csv('./ForestCover.csv')

In [34]:
data.shape

(286048, 11)

In [35]:
pd.pivot_table(data,
             values = 'Col2',
               index = 'y', 
              aggfunc = 'count')

,Col2
y,
0,283301
1,2747


In [36]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,y
0,2804,139,9,268,65,3180,234,238,135,6121,0
1,2785,155,18,242,118,3090,238,238,122,6211,0
2,2579,132,6,300,-15,67,230,237,140,6031,0
3,2886,151,11,371,26,5253,234,240,136,4051,0
4,2742,134,22,150,69,3215,248,224,92,6091,0


### Autoencoders

In [37]:
features = data.drop(columns = ['y'])
target = data['y']

In [38]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [39]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(5,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [40]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [41]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=512,
)

end = time.process_time()
forestcover_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
forestcover_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
559/559 [==============================] - 1s 1ms/step - loss: 0.0223 - mse: 0.0457
Epoch 2/20
559/559 [==============================] - 1s 1ms/step - loss: 0.0129 - mse: 0.0261
Epoch 3/20
559/559 [==============================] - 1s 1ms/step - loss: 0.0104 - mse: 0.0212
Epoch 4/20
559/559 [==============================] - 1s 1ms/step - loss: 0.0092 - mse: 0.0187
Epoch 5/20
559/559 [==============================] - 1s 1ms/step - loss: 0.0085 - mse: 0.0173
Epoch 6/20
559/559 [==============================] - 1s 1ms/step - loss: 0.0081 - mse: 0.0163
Epoch 7/20
559/559 [==============================] - 1s 1ms/step - loss: 0.0078 - mse: 0.0157
Epoch 8/20
559/559 [==============================] - 1s 2ms/step - loss: 0.0076 - mse: 0.0152
Epoch 9/20
559/559 [==============================] - 1s 1ms/step - loss: 0.0075 - mse: 0.0149
Epoch 10/20
559/559 [==============================] - 1s 1ms/step - loss: 0.0061 - mse: 0.0122
Epoch 11/20
559/559 [============================

In [42]:
confusion_matrix(target, predictions)

array([[258759,  24542],
       [    40,   2707]], dtype=int64)

In [43]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
forestcover_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9862151064866739

In [44]:
forestcover_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95    283301
           1       0.10      0.99      0.18      2747

    accuracy                           0.91    286048
   macro avg       0.55      0.95      0.57    286048
weighted avg       0.99      0.91      0.95    286048



In [45]:
precision, recall, thresholds = precision_recall_curve(target, scores)
forestcover_ae_auc_precision_recall = metrics.auc(recall, precision)
print(forestcover_ae_auc_precision_recall)

0.3536141935677184


## Annthyroid

**Dataset source**: http://odds.cs.stonybrook.edu/annthyroid-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Aditional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

In [46]:
data = pd.read_csv('./annthyroid.csv')

In [47]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,y
0,0.73,0.00060,0.015,0.120,0.082,0.146,0
1,0.24,0.00025,0.030,0.143,0.133,0.108,0
2,0.47,0.00190,0.024,0.102,0.131,0.078,0
3,0.64,0.00090,0.017,0.077,0.090,0.085,0
4,0.23,0.00025,0.026,0.139,0.090,0.153,0


In [48]:
data.shape

(7200, 7)

In [49]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,6666
1,534


### Autoencoders

In [50]:
features = data.drop(columns = ['y'])
target = data['y']

In [51]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [52]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(3,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [53]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [54]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=512,
)

end = time.process_time()
annthyroid_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
annthyroid_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
15/15 [==============================] - 1s 2ms/step - loss: 0.0690 - mse: 0.1156
Epoch 2/20
15/15 [==============================] - 0s 2ms/step - loss: 0.0673 - mse: 0.1123
Epoch 3/20
15/15 [==============================] - 0s 2ms/step - loss: 0.0657 - mse: 0.1093
Epoch 4/20
15/15 [==============================] - 0s 2ms/step - loss: 0.0643 - mse: 0.1065
Epoch 5/20
15/15 [==============================] - 0s 2ms/step - loss: 0.0630 - mse: 0.1041
Epoch 6/20
15/15 [==============================] - 0s 2ms/step - loss: 0.0619 - mse: 0.1020
Epoch 7/20
15/15 [==============================] - 0s 2ms/step - loss: 0.0609 - mse: 0.1001
Epoch 8/20
15/15 [==============================] - 0s 2ms/step - loss: 0.0599 - mse: 0.0982
Epoch 9/20
15/15 [==============================] - 0s 2ms/step - loss: 0.0589 - mse: 0.0963
Epoch 10/20
15/15 [==============================] - 0s 2ms/step - loss: 0.0578 - mse: 0.0943
Epoch 11/20
15/15 [==============================] - 0s 2ms/step - lo

In [55]:
confusion_matrix(target, predictions)

array([[5753,  913],
       [ 455,   79]], dtype=int64)

In [56]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
annthyroid_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5082216086777217

In [57]:
annthyroid_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.93      0.86      0.89      6666
           1       0.08      0.15      0.10       534

    accuracy                           0.81      7200
   macro avg       0.50      0.51      0.50      7200
weighted avg       0.86      0.81      0.84      7200



In [58]:
precision, recall, thresholds = precision_recall_curve(target, scores)
annthyroid_ae_auc_precision_recall = metrics.auc(recall, precision)
print(annthyroid_ae_auc_precision_recall)

0.08391467270909059


## Credit card

**Dataset source**: https://www.kaggle.com/mlg-ulb/creditcardfraud

**Additional sources:**

Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon

Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE

Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)

Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier

Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing

Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019

Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019

Yann-Aël Le Borgne, Gianluca Bontempi Machine Learning for Credit Card Fraud Detection - Practical Handbook

In [59]:
data = pd.read_csv('./creditcard.csv')

In [60]:
data = data.drop(columns = ['Time'])

In [61]:
data.shape

(284807, 30)

In [62]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [63]:
pd.pivot_table(data,
             values = 'V1',
               index = 'Class', 
              aggfunc = 'count')

,V1
Class,
0,284315
1,492


### Autoencoders

In [64]:
features = data.drop(columns = ['Class'])
target = data['Class']

In [65]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [66]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(14,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [67]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy()) 
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [68]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=512,
)

end = time.process_time()
creditcard_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

#end = time.process_time()
end = time.process_time()
creditcard_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
557/557 [==============================] - 2s 2ms/step - loss: 0.0035 - mse: 0.0080
Epoch 2/20
557/557 [==============================] - 1s 2ms/step - loss: 0.0011 - mse: 0.0023
Epoch 3/20
557/557 [==============================] - 1s 2ms/step - loss: 8.9176e-04 - mse: 0.0019
Epoch 4/20
557/557 [==============================] - 2s 3ms/step - loss: 7.9054e-04 - mse: 0.0017
Epoch 5/20
557/557 [==============================] - 2s 3ms/step - loss: 7.0599e-04 - mse: 0.0015
Epoch 6/20
557/557 [==============================] - 1s 2ms/step - loss: 6.3177e-04 - mse: 0.0014
Epoch 7/20
557/557 [==============================] - 1s 2ms/step - loss: 5.7075e-04 - mse: 0.0013
Epoch 8/20
557/557 [==============================] - 1s 2ms/step - loss: 5.2614e-04 - mse: 0.0012
Epoch 9/20
557/557 [==============================] - 1s 2ms/step - loss: 5.0262e-04 - mse: 0.0011
Epoch 10/20
557/557 [==============================] - 1s 2ms/step - loss: 4.9001e-04 - mse: 0.0011
Epoch 11/20
557/5

In [69]:
confusion_matrix(target, predictions)

array([[279657,   4658],
       [    79,    413]], dtype=int64)

In [70]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
creditcard_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9565936470612795

In [71]:
creditcard_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    284315
           1       0.08      0.84      0.15       492

    accuracy                           0.98    284807
   macro avg       0.54      0.91      0.57    284807
weighted avg       1.00      0.98      0.99    284807



In [72]:
precision, recall, thresholds = precision_recall_curve(target, scores)
creditcard_ae_auc_precision_recall = metrics.auc(recall, precision)
print(creditcard_ae_auc_precision_recall)

0.3623062982636414


## Mammography

**Dataset source**: http://odds.cs.stonybrook.edu/mammography-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

In [73]:
data = pd.read_csv('./mammography.csv')

In [74]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,y
0,0.230020,5.072578,-0.276061,0.832444,-0.377866,0.480322,0
1,0.155491,-0.169390,0.670652,-0.859553,-0.377866,-0.945723,0
2,-0.784415,-0.443654,5.674705,-0.859553,-0.377866,-0.945723,0
3,0.546088,0.131415,-0.456387,-0.859553,-0.377866,-0.945723,0
4,-0.102987,-0.394994,-0.140816,0.979703,-0.377866,1.013566,0


In [75]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,10923
1,260


In [76]:
data.shape

(11183, 7)

### Autoencoders

In [77]:
features = data.drop(columns = ['y'])
target = data['y']

In [78]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [79]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(3,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [80]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [81]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=512,
)

end = time.process_time()
mammography_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
mammography_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
22/22 [==============================] - 1s 3ms/step - loss: 0.1371 - mse: 0.2168
Epoch 2/20
22/22 [==============================] - 0s 3ms/step - loss: 0.1334 - mse: 0.2097
Epoch 3/20
22/22 [==============================] - 0s 3ms/step - loss: 0.1300 - mse: 0.2032
Epoch 4/20
22/22 [==============================] - 0s 2ms/step - loss: 0.1268 - mse: 0.1972
Epoch 5/20
22/22 [==============================] - 0s 2ms/step - loss: 0.1238 - mse: 0.1915
Epoch 6/20
22/22 [==============================] - 0s 3ms/step - loss: 0.1209 - mse: 0.1863
Epoch 7/20
22/22 [==============================] - 0s 2ms/step - loss: 0.1182 - mse: 0.1812
Epoch 8/20
22/22 [==============================] - 0s 2ms/step - loss: 0.1156 - mse: 0.1765
Epoch 9/20
22/22 [==============================] - 0s 2ms/step - loss: 0.1131 - mse: 0.1720
Epoch 10/20
22/22 [==============================] - 0s 3ms/step - loss: 0.1106 - mse: 0.1677
Epoch 11/20
22/22 [==============================] - 0s 2ms/step - lo

In [82]:
confusion_matrix(target, predictions)

array([[7580, 3343],
       [ 253,    7]], dtype=int64)

In [83]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
mammography_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.2301952830653737

In [84]:
mammography_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.97      0.69      0.81     10923
           1       0.00      0.03      0.00       260

    accuracy                           0.68     11183
   macro avg       0.48      0.36      0.41     11183
weighted avg       0.95      0.68      0.79     11183



In [85]:
precision, recall, thresholds = precision_recall_curve(target, scores)
mammography_ae_auc_precision_recall = metrics.auc(recall, precision)
print(mammography_ae_auc_precision_recall)

0.027235123685653048


## Shuttle

**Dataset source**: http://odds.cs.stonybrook.edu/shuttle-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Abe, Naoki, Bianca Zadrozny, and John Langford. “Outlier detection by active learning.” Proceedings of the 12th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2006.

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

Kai Ming Ting, Guang-Tong Zhou, Fei Tony Liu & Tan Swee Chuan. (2010). Mass Estimation and Its Applications. Proceedings of The 16th ACM SIGKDD Conference on Knowledge Discovery and Data Mining 2010. pp. 989-998.

Swee Chuan Tan, Kai Ming Ting & Fei Tony Liu. (2011). Fast Anomaly Detection for Streaming Data. Proceedings of the International Joint Conference on Artificial Intelligence 2011. pp.1151-1156.

In [86]:
data = pd.read_csv('./shuttle.csv', sep = ',')

In [87]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,y
0,50,21,77,0,28,0,27,48,22,1
1,53,0,82,0,52,-5,29,30,2,0
2,37,0,76,0,28,18,40,48,8,0
3,37,0,79,0,34,-26,43,46,2,0
4,85,0,88,-4,6,1,3,83,80,1


In [88]:
data.shape

(49097, 10)

In [89]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,45586
1,3511


### Autoencoders

In [90]:
features = data.drop(columns = ['y'])
target = data['y']

In [91]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features .copy())

In [92]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(5,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [93]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [94]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=512,
)

end = time.process_time()
shuttle_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
shuttle_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
96/96 [==============================] - 1s 2ms/step - loss: 0.0150 - mse: 0.0301
Epoch 2/20
96/96 [==============================] - 0s 2ms/step - loss: 0.0093 - mse: 0.0173
Epoch 3/20
96/96 [==============================] - 0s 2ms/step - loss: 0.0068 - mse: 0.0123
Epoch 4/20
96/96 [==============================] - 0s 2ms/step - loss: 0.0055 - mse: 0.0098
Epoch 5/20
96/96 [==============================] - 0s 2ms/step - loss: 0.0047 - mse: 0.0083
Epoch 6/20
96/96 [==============================] - 0s 2ms/step - loss: 0.0040 - mse: 0.0071
Epoch 7/20
96/96 [==============================] - 0s 2ms/step - loss: 0.0034 - mse: 0.0060
Epoch 8/20
96/96 [==============================] - 0s 2ms/step - loss: 0.0027 - mse: 0.0050
Epoch 9/20
96/96 [==============================] - 0s 2ms/step - loss: 0.0021 - mse: 0.0041
Epoch 10/20
96/96 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0036
Epoch 11/20
96/96 [==============================] - ETA: 0s - loss: 

In [95]:
confusion_matrix(target, predictions)

array([[45048,   538],
       [  178,  3333]], dtype=int64)

In [96]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
shuttle_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9829925123418607

In [97]:
shuttle_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     45586
           1       0.86      0.95      0.90      3511

    accuracy                           0.99     49097
   macro avg       0.93      0.97      0.95     49097
weighted avg       0.99      0.99      0.99     49097



In [98]:
precision, recall, thresholds = precision_recall_curve(target, scores)
shuttle_ae_auc_precision_recall = metrics.auc(recall, precision)
print(shuttle_ae_auc_precision_recall)

0.8668594624804314


## mnist

**Dataset source**: http://odds.cs.stonybrook.edu/mnist-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Bandaragoda, Tharindu R., et al. “Efficient Anomaly Detection by Isolation Using Nearest Neighbour Ensemble.” 2014 IEEE International Conference on Data Mining Workshop. IEEE, 2014.

In [99]:
data = pd.read_csv('./mnist.csv')

In [100]:
data = data.drop(columns = ['Col1','Col4', 'Col7', 'Col22', 'Col27', 'Col29', 'Col38', 'Col41', 'Col51', 'Col53', 'Col54', 'Col61', 'Col62', 'Col71', 'Col73', 'Col79', 'Col87', 'Col88', 'Col89', 'Col90',
'Col92', 'Col100'])

In [101]:
pd.pivot_table(data,
             values = 'Col2',
               index = 'y', 
              aggfunc = 'count')

,Col2
y,
0,6903
1,700


In [102]:
data.shape

(7603, 79)

In [103]:
data.head()

,Col2,Col3,Col5,Col6,Col8,Col9,Col10,Col11,Col12,Col13,...,Col86,Col91,Col93,Col94,Col95,Col96,Col97,Col98,Col99,y
0,-73.804153,198.205963,-13.124617,-1.1501,-0.141633,179.249390,114.661163,-80.736702,130.659348,162.649841,...,-15.392716,188.055649,-4.469967,158.381409,-137.100632,27.131416,-2.274633,-0.00065,-12.351267,0
1,-73.804153,197.205963,-13.124617,-1.1501,-0.141633,179.249390,-44.338833,-80.736702,128.659348,190.649841,...,-15.392716,186.055649,-4.469967,123.381416,-137.100632,157.131409,-2.274633,-0.00065,-12.351267,0
2,-73.804153,-53.794033,-13.124617,-1.1501,-0.141633,-73.750618,-44.338833,170.263306,130.659348,46.649849,...,-15.392716,188.055649,-4.469967,157.381409,-137.100632,-93.868584,-2.274633,-0.00065,-12.351267,0
3,-73.804153,86.205963,-13.124617,-1.1501,-0.141633,76.249382,208.661163,107.263298,130.659348,190.649841,...,-15.392716,188.055649,-4.469967,157.381409,-137.100632,74.131416,-2.274633,-0.00065,-12.351267,0
4,-27.804153,199.205963,-13.124617,-1.1501,-0.141633,179.249390,-44.338833,-80.736702,130.659348,91.649849,...,-15.392716,188.055649,-4.469967,22.381416,-137.100632,159.131409,-2.274633,-0.00065,-12.351267,0


### Autoencoders

In [104]:
features = data.drop(columns = ['y'])
target = data['y']

In [105]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [106]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(39,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [107]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [108]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=512,
)

end = time.process_time()
mnist_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
mnist_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
15/15 [==============================] - 1s 4ms/step - loss: 0.1250 - mse: 0.2119
Epoch 2/20
15/15 [==============================] - 0s 4ms/step - loss: 0.1047 - mse: 0.1768
Epoch 3/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0787 - mse: 0.1355
Epoch 4/20
15/15 [==============================] - 0s 3ms/step - loss: 0.0564 - mse: 0.1028
Epoch 5/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0455 - mse: 0.0876
Epoch 6/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0412 - mse: 0.0811
Epoch 7/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0386 - mse: 0.0766
Epoch 8/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0364 - mse: 0.0723
Epoch 9/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0345 - mse: 0.0689
Epoch 10/20
15/15 [==============================] - 0s 4ms/step - loss: 0.0327 - mse: 0.0652
Epoch 11/20
15/15 [==============================] - 0s 4ms/step - lo

In [109]:
confusion_matrix(target, predictions)

array([[6171,  732],
       [ 312,  388]], dtype=int64)

In [110]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
mnist_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.8577279029821402

In [111]:
mnist_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.95      0.89      0.92      6903
           1       0.35      0.55      0.43       700

    accuracy                           0.86      7603
   macro avg       0.65      0.72      0.67      7603
weighted avg       0.90      0.86      0.88      7603



In [112]:
precision, recall, thresholds = precision_recall_curve(target, scores)
mnist_ae_auc_precision_recall = metrics.auc(recall, precision)
print(mnist_ae_auc_precision_recall)

0.3410197523364251


## vowels

**Dataset source**: http://odds.cs.stonybrook.edu/japanese-vowels-data/

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [113]:
data = pd.read_csv('./vowels.csv')

In [114]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0.0,1406
1.0,50


In [115]:
data.shape

(1456, 13)

In [116]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,y
0,0.580469,-0.902534,0.617899,-0.997942,-2.463799,-0.846455,2.349849,0.375400,-0.649334,1.604637,-0.623060,-0.383125,0.0
1,0.784375,-1.077366,0.615781,-0.921911,-2.388553,-0.638047,2.106684,0.361018,-0.714317,1.260236,-0.423339,-0.287791,0.0
2,0.791292,-1.086242,0.669773,-0.806112,-2.260781,-0.538491,2.053282,0.266492,-0.842815,1.081797,-0.267201,-0.172203,0.0
3,1.217306,-1.083425,0.855483,-0.724879,-2.155552,-0.101879,1.768597,0.303151,-1.044710,0.655290,0.214298,-0.341840,0.0
4,1.065352,-1.030178,0.773297,-0.452289,-1.955907,0.248205,1.530474,0.253740,-0.968961,-0.208287,0.331578,0.007288,0.0


### Autoencoders

In [117]:
features = data.drop(columns = ['y'])
target = data['y']

In [118]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [119]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(6, input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [120]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [121]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=64,
)

end = time.process_time()
vowels_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
vowels_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
23/23 [==============================] - 1s 2ms/step - loss: 0.0176 - mse: 0.0363
Epoch 2/20
23/23 [==============================] - 0s 2ms/step - loss: 0.0168 - mse: 0.0349
Epoch 3/20
23/23 [==============================] - 0s 2ms/step - loss: 0.0161 - mse: 0.0336
Epoch 4/20
23/23 [==============================] - 0s 2ms/step - loss: 0.0157 - mse: 0.0329
Epoch 5/20
23/23 [==============================] - 0s 2ms/step - loss: 0.0153 - mse: 0.0322
Epoch 6/20
23/23 [==============================] - 0s 3ms/step - loss: 0.0151 - mse: 0.0317
Epoch 7/20
23/23 [==============================] - 0s 3ms/step - loss: 0.0148 - mse: 0.0312
Epoch 8/20
23/23 [==============================] - 0s 2ms/step - loss: 0.0146 - mse: 0.0308
Epoch 9/20
23/23 [==============================] - 0s 3ms/step - loss: 0.0144 - mse: 0.0304
Epoch 10/20
23/23 [==============================] - 0s 3ms/step - loss: 0.0142 - mse: 0.0298
Epoch 11/20
23/23 [==============================] - 0s 3ms/step - lo

In [122]:
confusion_matrix(target, predictions)

array([[1213,  193],
       [  37,   13]], dtype=int64)

In [123]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
vowels_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.5675960170697012

In [124]:
vowels_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.97      0.86      0.91      1406
           1       0.06      0.26      0.10        50

    accuracy                           0.84      1456
   macro avg       0.52      0.56      0.51      1456
weighted avg       0.94      0.84      0.89      1456



In [125]:
precision, recall, thresholds = precision_recall_curve(target, scores)
vowels_ae_auc_precision_recall = metrics.auc(recall, precision)
print(vowels_ae_auc_precision_recall)

0.041917486963610945


## Seismic

**Dataset source**: http://odds.cs.stonybrook.edu/seismic-dataset/ (data is transformed from .arff to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

Saket Sathe and Charu C. Aggarwal. LODES: Local Density meets Spectral Outlier Detection. SIAM Conference on Data Mining, 2016.

In [126]:
data = pd.read_csv('./seismic.csv', sep = ',')

In [127]:
data = data.drop(columns = ['nbumps6','nbumps7','nbumps89'])

In [128]:
data.shape

(2584, 16)

In [129]:
drop_enc = OneHotEncoder(drop='first').fit_transform(data[['seismic','seismoacoustic','shift','ghazard']])

In [130]:
cat_var = pd.DataFrame(drop_enc.toarray())
cat_var.columns = ['seismic: b', 'seismoacoustic: b','seismoacoustic: c','shift: W','ghazard: b','ghazard: c']

In [131]:
data = pd.concat([data, cat_var], axis = 1)

In [132]:
data = data.drop(columns = ['seismic','seismoacoustic','shift','ghazard'])

In [133]:
data.head()

,genergy,gpuls,gdenergy,gdpuls,nbumps,nbumps2,nbumps3,nbumps4,nbumps5,energy,maxenergy,class,seismic: b,seismoacoustic: b,seismoacoustic: c,shift: W,ghazard: b,ghazard: c
0,15180,48,-72,-72,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,14720,33,-70,-79,1,0,1,0,0,2000,2000,0,0.0,0.0,0.0,0.0,0.0,0.0
2,8050,30,-81,-78,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,28820,171,-23,40,1,0,1,0,0,3000,3000,0,0.0,0.0,0.0,0.0,0.0,0.0
4,12640,57,-63,-52,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
pd.pivot_table(data,
             values = 'genergy',
               index = 'class', 
              aggfunc = 'count')

,genergy
class,
0,2414
1,170


### Autoencoders

In [135]:
features = data.drop(columns = ['class'])
target = data['class']

In [136]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [137]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(10,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [138]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [139]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=64,
)

end = time.process_time()
seismic_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
seismic_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
41/41 [==============================] - 1s 2ms/step - loss: 0.1325 - mse: 0.2142
Epoch 2/20
41/41 [==============================] - 0s 2ms/step - loss: 0.1190 - mse: 0.1901
Epoch 3/20
41/41 [==============================] - 0s 2ms/step - loss: 0.0993 - mse: 0.1562
Epoch 4/20
41/41 [==============================] - 0s 2ms/step - loss: 0.0764 - mse: 0.1187
Epoch 5/20
41/41 [==============================] - 0s 2ms/step - loss: 0.0568 - mse: 0.0889
Epoch 6/20
41/41 [==============================] - 0s 2ms/step - loss: 0.0442 - mse: 0.0713
Epoch 7/20
41/41 [==============================] - 0s 2ms/step - loss: 0.0368 - mse: 0.0610
Epoch 8/20
41/41 [==============================] - 0s 2ms/step - loss: 0.0320 - mse: 0.0543
Epoch 9/20
41/41 [==============================] - 0s 2ms/step - loss: 0.0290 - mse: 0.0500
Epoch 10/20
41/41 [==============================] - 0s 2ms/step - loss: 0.0263 - mse: 0.0459
Epoch 11/20
41/41 [==============================] - 0s 2ms/step - lo

In [140]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
seismic_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.6356133339831376

In [141]:
seismic_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.94      0.87      0.90      2414
           1       0.11      0.23      0.15       170

    accuracy                           0.82      2584
   macro avg       0.52      0.55      0.52      2584
weighted avg       0.89      0.82      0.85      2584



In [142]:
precision, recall, thresholds = precision_recall_curve(target, scores)
seismic_ae_auc_precision_recall = metrics.auc(recall, precision)
print(seismic_ae_auc_precision_recall)

0.11008463204663357


## Musk

**Dataset source**: http://odds.cs.stonybrook.edu/musk-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources:**

C. C. Aggarwal and S. Sathe, “Theoretical foundations and algorithms for outlier ensembles.” ACM SIGKDD Explorations Newsletter, vol. 17, no. 1, pp. 24–47, 2015.

In [143]:
data = pd.read_csv('./musk.csv', sep = ',')

In [144]:
data.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,...,Col158,Col159,Col160,Col161,Col162,Col163,Col164,Col165,Col166,y
0,46.0,-108.0,-60.0,-69.0,-117.0,49.0,38.0,-161.0,-8.0,5.0,...,-308.0,52.0,-7.0,39.0,126.0,156.0,-50.0,-112.0,96.0,1.0
1,41.0,-188.0,-145.0,22.0,-117.0,-6.0,57.0,-171.0,-39.0,-100.0,...,-59.0,-2.0,52.0,103.0,136.0,169.0,-61.0,-136.0,79.0,1.0
2,46.0,-194.0,-145.0,28.0,-117.0,73.0,57.0,-168.0,-39.0,-22.0,...,-134.0,-154.0,57.0,143.0,142.0,165.0,-67.0,-145.0,39.0,1.0
3,41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,-170.0,-39.0,-99.0,...,-60.0,-4.0,52.0,104.0,136.0,168.0,-60.0,-135.0,80.0,1.0
4,41.0,-188.0,-145.0,22.0,-117.0,-7.0,57.0,-170.0,-39.0,-99.0,...,-60.0,-4.0,52.0,104.0,137.0,168.0,-60.0,-135.0,80.0,1.0


In [145]:
data['y'] = data['y'].astype(int)

In [146]:
data.shape

(3062, 167)

In [147]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,2965
1,97


### Autoencoders

In [148]:
features = data.drop(columns = ['y'])
target = data['y']

In [149]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [150]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(83,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [151]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [152]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=64,
)

end = time.process_time()
musk_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
musk_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
48/48 [==============================] - 1s 3ms/step - loss: 0.0303 - mse: 0.0577
Epoch 2/20
48/48 [==============================] - 0s 3ms/step - loss: 0.0154 - mse: 0.0314
Epoch 3/20
48/48 [==============================] - 0s 3ms/step - loss: 0.0120 - mse: 0.0247
Epoch 4/20
48/48 [==============================] - 0s 3ms/step - loss: 0.0096 - mse: 0.0200
Epoch 5/20
48/48 [==============================] - 0s 3ms/step - loss: 0.0080 - mse: 0.0168
Epoch 6/20
48/48 [==============================] - 0s 3ms/step - loss: 0.0070 - mse: 0.0148
Epoch 7/20
48/48 [==============================] - 0s 3ms/step - loss: 0.0063 - mse: 0.0133
Epoch 8/20
48/48 [==============================] - 0s 3ms/step - loss: 0.0058 - mse: 0.0123
Epoch 9/20
48/48 [==============================] - 0s 3ms/step - loss: 0.0054 - mse: 0.0114
Epoch 10/20
48/48 [==============================] - 0s 3ms/step - loss: 0.0051 - mse: 0.0108
Epoch 11/20
48/48 [==============================] - 0s 3ms/step - lo

In [153]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
musk_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.9990299195076581

In [154]:
musk_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97      2965
           1       0.37      1.00      0.54        97

    accuracy                           0.95      3062
   macro avg       0.69      0.97      0.76      3062
weighted avg       0.98      0.95      0.96      3062



In [155]:
precision, recall, thresholds = precision_recall_curve(target, scores)
musk_ae_auc_precision_recall = metrics.auc(recall, precision)
print(musk_ae_auc_precision_recall)

0.9701428296065564


## bank

**Dataset source**: https://github.com/GuansongPang/ADRepository-Anomaly-detection-datasets/tree/main/categorical%20data

Pang, G., Shen, C., Cao, L., & Hengel, A. V. D. (2021). Deep learning for anomaly detection: A review. ACM Computing Surveys (CSUR), 54(2), 1-38.

In [156]:
data = pd.read_csv('./bank.csv')

In [157]:
data.head()

,age,job=housemaid,job=services,job=admin.,job=blue-collar,job=technician,job=retired,job=management,job=unemployed,job=self-employed,...,previous,poutcome=nonexistent,poutcome=failure,poutcome=success,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,class
0,0.209877,0,0,0,0,0,0,0,0,0,...,0.000000,1,0,0,1.000000,0.882307,0.376569,0.980730,1.000000,0
1,0.296296,0,0,1,0,0,0,0,0,0,...,0.000000,1,0,0,1.000000,0.484412,0.615063,0.981183,1.000000,0
2,0.246914,1,0,0,0,0,0,0,0,0,...,0.000000,1,0,0,0.937500,0.698753,0.602510,0.957379,0.859735,0
3,0.160494,0,1,0,0,0,0,0,0,0,...,0.142857,0,1,0,0.333333,0.269680,0.192469,0.150759,0.512287,0
4,0.530864,0,0,0,1,0,0,0,0,0,...,0.000000,1,0,0,0.333333,0.340608,0.154812,0.174790,0.512287,1


In [158]:
pd.pivot_table(data,
             values = 'age',
               index = 'class', 
              aggfunc = 'count')

,age
class,
0,36548
1,4640


### Autoencoders

In [159]:
features = data.drop(columns = ['class'])
target = data['class']

In [160]:
# min max scale the input data
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(features.copy())

In [161]:
class AutoEncoder(Model):

    def __init__(self, output_units):
        super().__init__()
        self.encoder = Sequential([
          Dense(31,input_dim = output_units, activation='relu'),
          Dropout(0.1),
        ])
        self.decoder = Sequential([
          Dense(output_units, activation='sigmoid')
        ])
  
    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

In [162]:
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
  # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_train_scaled, threshold):
    predictions = model.predict(x_train_scaled)
  # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_train_scaled)
  # 1 = anomaly, 0 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 1.0 if x == True else 0.0)
    return preds, errors

In [163]:
start = time.process_time()

model = AutoEncoder(output_units=x_train_scaled.shape[1])

model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=512,
)

end = time.process_time()
bank_autoencoders_train_time = end - start
print(end - start)

start = time.process_time()

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")

predictions, scores = get_predictions(model, x_train_scaled, threshold)

end = time.process_time()
bank_autoencoders_test_time = end - start
print(end - start)

Epoch 1/20
81/81 [==============================] - 1s 4ms/step - loss: 0.0877 - mse: 0.1550
Epoch 2/20
81/81 [==============================] - 0s 4ms/step - loss: 0.0424 - mse: 0.0890
Epoch 3/20
81/81 [==============================] - 0s 4ms/step - loss: 0.0346 - mse: 0.0736
Epoch 4/20
81/81 [==============================] - 0s 4ms/step - loss: 0.0285 - mse: 0.0607
Epoch 5/20
81/81 [==============================] - 0s 4ms/step - loss: 0.0238 - mse: 0.0508
Epoch 6/20
81/81 [==============================] - 0s 4ms/step - loss: 0.0198 - mse: 0.0424
Epoch 7/20
81/81 [==============================] - 0s 4ms/step - loss: 0.0167 - mse: 0.0357
Epoch 8/20
81/81 [==============================] - 0s 4ms/step - loss: 0.0145 - mse: 0.0311
Epoch 9/20
81/81 [==============================] - 0s 4ms/step - loss: 0.0129 - mse: 0.0276
Epoch 10/20
81/81 [==============================] - 0s 4ms/step - loss: 0.0118 - mse: 0.0251
Epoch 11/20
81/81 [==============================] - 0s 4ms/step - lo

In [164]:
fpr, tpr, _ = metrics.roc_curve(target, scores)
bank_autoencoders_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

0.7227007563034724

In [165]:
bank_autoencoders_report = classification_report(target, predictions, target_names = ['0','1'], output_dict = True)
print(classification_report(target, predictions, target_names = ['0','1']))

              precision    recall  f1-score   support

           0       0.92      0.84      0.87     36548
           1       0.24      0.40      0.29      4640

    accuracy                           0.79     41188
   macro avg       0.58      0.62      0.58     41188
weighted avg       0.84      0.79      0.81     41188



In [166]:
precision, recall, thresholds = precision_recall_curve(target, scores)
bank_ae_auc_precision_recall = metrics.auc(recall, precision)
print(bank_ae_auc_precision_recall)

0.25835712844212444


## Performance

In [176]:
performance = pd.DataFrame(columns = ['F1 score', 'recall', 'precision', 'AUC', 'AUPRC', 
                                      'Training time','Inference time','Total time'])

In [177]:
f1_score_ae = {'arrhythmia':arrhythmia_autoencoders_report['1']['f1-score'],
                       'cardio':cardio_autoencoders_report['1']['f1-score'], 
                        'forestcover':forestcover_autoencoders_report['1']['f1-score'], 
                       'annthyroid':annthyroid_autoencoders_report['1']['f1-score'],       
                        'creditcard':creditcard_autoencoders_report['1']['f1-score'], 
                       'mammography':mammography_autoencoders_report['1']['f1-score'], 
                        'shuttle':shuttle_autoencoders_report['1']['f1-score'], 
                      'mnist':mnist_autoencoders_report['1']['f1-score'], 
                  'vowels':vowels_autoencoders_report['1']['f1-score'], 
                  'seismic':seismic_autoencoders_report['1']['f1-score'], 
                  'musk':musk_autoencoders_report['1']['f1-score'], 
                  'bank':bank_autoencoders_report['1']['f1-score']}
f1_score_ae_df = pd.DataFrame.from_dict(f1_score_ae, orient='index', columns = ['F1 score']).reset_index()

In [178]:
recall_ae = {'arrhythmia':arrhythmia_autoencoders_report['1']['recall'],
                       'cardio':cardio_autoencoders_report['1']['recall'], 
                        'forestcover':forestcover_autoencoders_report['1']['recall'], 
                       'annthyroid':annthyroid_autoencoders_report['1']['recall'],       
                        'creditcard':creditcard_autoencoders_report['1']['recall'], 
                       'mammography':mammography_autoencoders_report['1']['recall'], 
                        'shuttle':shuttle_autoencoders_report['1']['recall'], 
                      'mnist':mnist_autoencoders_report['1']['recall'], 
                  'vowels':vowels_autoencoders_report['1']['recall'], 
                  'seismic':seismic_autoencoders_report['1']['recall'], 
                  'musk':musk_autoencoders_report['1']['recall'], 
                  'bank':bank_autoencoders_report['1']['recall'], }
recall_ae_df = pd.DataFrame.from_dict(recall_ae, orient='index', columns = ['Recall']).reset_index()

In [179]:
precision_ae = {'arrhythmia':arrhythmia_autoencoders_report['1']['precision'],
                       'cardio':cardio_autoencoders_report['1']['precision'], 
                        'forestcover':forestcover_autoencoders_report['1']['precision'], 
                       'annthyroid':annthyroid_autoencoders_report['1']['precision'],       
                        'creditcard':creditcard_autoencoders_report['1']['precision'], 
                       'mammography':mammography_autoencoders_report['1']['precision'], 
                        'shuttle':shuttle_autoencoders_report['1']['precision'], 
                      'mnist':mnist_autoencoders_report['1']['precision'], 
                  'vowels':vowels_autoencoders_report['1']['precision'], 
                  'seismic':seismic_autoencoders_report['1']['precision'], 
                  'musk':musk_autoencoders_report['1']['precision'], 
                  'bank':bank_autoencoders_report['1']['precision'], }
precision_ae_df = pd.DataFrame.from_dict(precision_ae, orient='index', columns = ['Precision']).reset_index()

In [180]:
auc_ae = {'arrhythmia':arrhythmia_autoencoders_auc,
                       'cardio':cardio_autoencoders_auc, 
                        'forestcover':forestcover_autoencoders_auc, 
                       'annthyroid':annthyroid_autoencoders_auc,       
                        'creditcard':creditcard_autoencoders_auc, 
                       'mammography':mammography_autoencoders_auc, 
                        'shuttle':shuttle_autoencoders_auc, 
                      'mnist':mnist_autoencoders_auc, 
                  'vowels':vowels_autoencoders_auc, 
                  'seismic':seismic_autoencoders_auc, 
                  'musk':musk_autoencoders_auc, 
                  'bank':bank_autoencoders_auc}
auc_ae_df = pd.DataFrame.from_dict(auc_ae, orient='index', columns = ['AUC']).reset_index()

In [181]:
auprc_ae = {'arrhythmia':arrhythmia_ae_auc_precision_recall,
                       'cardio':cardio_ae_auc_precision_recall, 
                        'forestcover':forestcover_ae_auc_precision_recall, 
                       'annthyroid':annthyroid_ae_auc_precision_recall,       
                        'creditcard':creditcard_ae_auc_precision_recall, 
                       'mammography':mammography_ae_auc_precision_recall, 
                        'shuttle':shuttle_ae_auc_precision_recall, 
                      'mnist':mnist_ae_auc_precision_recall, 
                  'vowels':vowels_ae_auc_precision_recall, 
                  'seismic':seismic_ae_auc_precision_recall, 
                  'musk':musk_ae_auc_precision_recall, 
                  'bank':bank_ae_auc_precision_recall}
auprc_ae_df = pd.DataFrame.from_dict(auprc_ae, orient='index', columns = ['AUPRC']).reset_index()

In [182]:
training_time_ae = {'arrhythmia':arrhythmia_autoencoders_train_time,
                       'cardio':cardio_autoencoders_train_time, 
                        'forestcover':forestcover_autoencoders_train_time, 
                       'annthyroid':annthyroid_autoencoders_train_time,       
                        'creditcard': creditcard_autoencoders_train_time, 
                       'mammography':mammography_autoencoders_train_time, 
                        'shuttle':shuttle_autoencoders_train_time, 
                      'mnist':mnist_autoencoders_train_time, 
                  'vowels':vowels_autoencoders_train_time, 
                  'seismic':seismic_autoencoders_train_time, 
                  'musk':musk_autoencoders_train_time, 
                  'bank':bank_autoencoders_train_time}
training_time_ae_df = pd.DataFrame.from_dict(training_time_ae, orient='index', columns = ['Training time']).reset_index()

In [183]:
test_time_ae = {'arrhythmia':arrhythmia_autoencoders_test_time,
                       'cardio':cardio_autoencoders_test_time, 
                        'forestcover':forestcover_autoencoders_test_time, 
                       'annthyroid':annthyroid_autoencoders_test_time,       
                        'creditcard':creditcard_autoencoders_test_time, 
                       'mammography':mammography_autoencoders_test_time, 
                        'shuttle':shuttle_autoencoders_test_time, 
                      'mnist':mnist_autoencoders_test_time, 
                  'vowels':vowels_autoencoders_test_time, 
                  'seismic':seismic_autoencoders_test_time, 
                  'musk':musk_autoencoders_test_time, 
                  'bank':bank_autoencoders_test_time}
test_time_ae_df = pd.DataFrame.from_dict(test_time_ae, orient='index', columns = ['Testing time']).reset_index()

In [184]:
total_time_ae = {'arrhythmia':arrhythmia_autoencoders_train_time + arrhythmia_autoencoders_test_time,
                       'cardio':cardio_autoencoders_train_time + cardio_autoencoders_test_time, 
                        'forestcover':forestcover_autoencoders_train_time + forestcover_autoencoders_test_time, 
                       'annthyroid':annthyroid_autoencoders_train_time + annthyroid_autoencoders_test_time,       
                        'creditcard': creditcard_autoencoders_train_time + creditcard_autoencoders_test_time, 
                       'mammography':mammography_autoencoders_train_time + mammography_autoencoders_test_time, 
                        'shuttle':shuttle_autoencoders_train_time + shuttle_autoencoders_test_time, 
                      'mnist':mnist_autoencoders_train_time + mnist_autoencoders_test_time, 
                  'vowels':vowels_autoencoders_train_time + vowels_autoencoders_test_time, 
                  'seismic':seismic_autoencoders_train_time + seismic_autoencoders_test_time, 
                  'musk':musk_autoencoders_train_time + musk_autoencoders_test_time, 
                  'bank':bank_autoencoders_train_time + bank_autoencoders_test_time}
total_time_ae_df = pd.DataFrame.from_dict(total_time_ae, orient='index', columns = ['Total time']).reset_index()

In [185]:
pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(f1_score_ae_df, recall_ae_df, how = 'inner'), 
                                    precision_ae_df, how ='inner'),
         auc_ae_df, how = 'inner'), auprc_ae_df, how = 'inner'), training_time_ae_df, how = 'inner'), 
         test_time_ae_df, how = 'inner'),total_time_ae_df, how = 'inner')

,index,F1 score,Recall,Precision,AUC,AUPRC,Training time,Testing time,Total time
0,arrhythmia,0.446281,0.409091,0.490909,0.802716,0.446179,2.046875,0.250000,2.296875
1,cardio,0.520095,0.625000,0.445344,0.898544,0.561593,3.187500,0.312500,3.500000
2,forestcover,0.180491,0.985439,0.099343,0.986215,0.353614,31.109375,21.671875,52.781250
3,annthyroid,0.103539,0.147940,0.079637,0.508222,0.083915,1.906250,0.812500,2.718750
4,creditcard,0.148481,0.839431,0.081444,0.956594,0.362306,48.828125,20.234375,69.062500
5,mammography,0.003878,0.026923,0.002090,0.230195,0.027235,2.703125,0.875000,3.578125
6,shuttle,0.903007,0.949302,0.861018,0.982993,0.866859,7.515625,4.671875,12.187500
7,mnist,0.426374,0.554286,0.346429,0.857728,0.341020,4.062500,1.015625,5.078125
8,vowels,0.101563,0.260000,0.063107,0.567596,0.041917,2.515625,0.406250,2.921875
9,seismic,0.146341,0.229412,0.107438,0.635613,0.110085,4.062500,0.453125,4.515625


In [ ]:
pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(f1_score_gan_df, recall_gan_df, how = 'inner'), 
                                    precision_gan_df, how ='inner'),
         auc_gan_df, how = 'inner'), auprc_gan_df, how = 'inner'), training_time_gan_df, how = 'inner'), 
         test_time_gan_df, how = 'inner'),total_time_gan_df, how = 'inner')